# Data Sourcing

In [154]:
pwd

'C:\\Users\\kyril\\code\\kyrilwittouck\\himalaya-project\\himalaya-project'

In [157]:
from data import Data

In [158]:
Data().get_data()

NameError: name 'csv_path' is not defined

In [88]:
import os
import pandas as pd

root_dir = os.path.abspath('')
xls_path = os.path.join(root_dir, 'data')
file_names = [f for f in os.listdir(xls_path) if f.endswith('.xls')]

def key_from_file_name(f):
    if f[-4:] == '.xls':
        return f[:-4]

# Create the dictionary
data = {}
for f in file_names:
    data[key_from_file_name(f)] = pd.read_excel(os.path.join(xls_path, f))
    
members = data['members']

In [137]:
data = members.copy()

In [116]:
data.head()

,exp_id,memb_id,peak_id,year,season,first_name,last_name,sex,birthdate,yob,...,death,death_date,death_time,death_type,death_height,death_class,summit_bid,summitter,hcn,mchksum
0,TKRG10101,1,TKRG,2010,1,David Alfred,Gottlieb,M,- -,1967.0,...,False,- -,NaN,0,0,0,5,1,0,2437753
1,TKRG10101,2,TKRG,2010,1,Julie,Hentrich,F,- -,1971.0,...,False,- -,NaN,0,0,0,1,17,0,2439311
2,TKRG10101,3,TKRG,2010,1,Joseph N. (Joe),Puryear,M,- -,1973.0,...,False,- -,NaN,0,0,0,5,1,0,2439937
3,AMPG10101,1,AMPG,2010,1,David John,Allen,M,- -,1961.0,...,False,- -,NaN,0,0,0,1,17,0,2435542
4,AMPG10101,2,AMPG,2010,1,David Charles,Barker,M,- -,1964.0,...,False,- -,NaN,0,0,0,1,17,0,2436719


# EDA

In [117]:
data.shape

(16383, 63)

In [138]:
col_to_drop = ['first_name', 
               'last_name',
               'age',
               'birthdate',
               'route2',
               'route3',
               'ascent2',
               'ascent3',
               'summit_time1', 
               'summit_time2', 
               'summit_time3',
               'summit_date2',
               'summit_date3',
               'o2_note',
               'death_time',
               'hcn',
               'mchksum']

data.drop(columns= col_to_drop, inplace = True)

In [153]:
for col in data:
    print(col, data[col].dtypes, data[col].isnull().sum())

exp_id object 0
memb_id int64 0
peak_id object 0
year int64 0
season int64 0
sex int64 0
yob float64 0
citizenship object 0
status object 0
residence object 0
occupation object 0
leader bool 0
deputy bool 0
bconly bool 0
nottobc bool 0
support bool 0
disabled bool 0
hired bool 0
sherpa bool 0
tibetan bool 0
summit_success bool 0
summit_claimed bool 0
summit_disputed bool 0
solo bool 0
traverse bool 0
ski bool 0
parapente bool 0
speed bool 0
highpt bool 0
high_point int64 0
summit_date1 datetime64[ns] 4211
route1 int64 0
ascent1 int64 0
o2_used bool 0
o2_none bool 0
o2_climb bool 0
o2_descent bool 0
o2_sleep bool 0
o2_medical bool 0
death bool 0
death_date datetime64[ns] 16228
death_type int64 0
death_height int64 0
death_class int64 0
summit_bid int64 0
summitter int64 0


In [120]:
data.head(3)

,exp_id,memb_id,peak_id,year,season,sex,yob,citizenship,status,residence,...,o2_descent,o2_sleep,o2_medical,death,death_date,death_type,death_height,death_class,summit_bid,summitter
0,TKRG10101,1,TKRG,2010,1,M,1967.0,USA,Leader,"Carlton, Washington",...,False,False,False,False,- -,0,0,0,5,1
1,TKRG10101,2,TKRG,2010,1,F,1971.0,USA,BC Manager,"Carlton, Washington",...,False,False,False,False,- -,0,0,0,1,17
2,TKRG10101,3,TKRG,2010,1,M,1973.0,USA,Climber,"Leavenworth, Washington",...,False,False,False,False,- -,0,0,0,5,1


# Data Preprocessing

In [150]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

data['summit_date1'] = pd.to_datetime(data.summit_date1, errors = 'coerce')
data['death_date'] = pd.to_datetime(data.death_date, errors = 'coerce')

le = LabelEncoder()
data['sex'] = le.fit_transform(data['sex'])

# Combine "age_transformer" and "One hot encoder" into a single preprocessor
preprocessor = ColumnTransformer([
    ('median', SimpleImputer(strategy='median'), ['yob']),
    ('frequency', SimpleImputer(strategy='most_frequent'), ['citizenship', 'status'])])

# Pass the combined preprocessor into a Pipeline as a single step
final_pipe = Pipeline([('preprocessing', preprocessor)])

data_trans = final_pipe.fit_transform(data)
data[['yob', 'citizenship', 'status']] = data_trans

In [151]:
data.occupation.fillna('Unknown', inplace = True)
data.residence.fillna('Unknown', inplace = True)